In [ ]:
#load rpy2 magic
%load_ext rpy2.ipython

# to switch off warning messages
import warnings
warnings.filterwarnings("ignore")

# make default cell width 85% of available screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# load R libraries & functions
%R options(warn=-1)
%R library(ggplot2)
%R library(gplots)
%R library(gridExtra)
%R library(scales)
%R library(ggrepel)
%R library(wesanderson)
%R library(reshape2)
%R library(dplyr)
%R library(plyr)
%R library(grid)

# load python modules
import glob
import re
import sys
import os
import sqlite3
import yaml
import seaborn as sns
import numpy as np
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

db = "./csvdb"

In [ ]:
# get options pipeline.yml
with open("pipeline.yml") as o:
    opts = yaml.load(o)
    
anndb = opts["annotations"]["database"]

In [ ]:
def fetch_DataFrame(query, dbhandle="csvdb"):
    '''Fetch query results and returns them as a pandas dataframe'''

    dbhandle = sqlite3.connect(dbhandle)

    cc = dbhandle.cursor()
    sqlresult = cc.execute(query).fetchall()
    cc.close()

    # see http://pandas.pydata.org/pandas-docs/dev/generated/
    # pandas.DataFrame.from_records.html#pandas.DataFrame.from_records
    # this method is design to handle sql_records with proper type
    # conversion

    field_names = [d[0] for d in cc.description]
    pandas_DataFrame = pd.DataFrame.from_records(
        sqlresult,
        columns=field_names
    )
    return pandas_DataFrame

In [ ]:
%%R 

# R functions
theme_notebook <- function(base_size=18, base_family="helvetica") {
                  (theme_set(theme_minimal(base_size=18))
                  + theme(plot.title = element_text(face="bold", size=20, hjust=0.5),
                             text = element_text(),
                             axis.title = element_text(face="bold",size = rel(1)),
                             axis.title.y = element_text(angle=90,vjust=2, size=20),
                             axis.title.x = element_text(vjust=-0.2, size=20),
                             axis.text = element_text(size=20),
                             axis.line = element_line(colour="black"),
                             axis.ticks = element_line(),
                             legend.key = element_rect(colour = NA),
                             legend.key.size= unit(0.5, "cm"),
                             legend.margin = unit(0.5, "cm"),
                             legend.text = element_text(size=14),
                             legend.title = element_text(size=16),
                             ))
}

# Set ggplot theme
theme_set(theme_notebook(base_size=18))

***
# Mapping QC
***

#### Sample information table:

In [ ]:
sample_info = fetch_DataFrame('''select * from sample_info''', db)
sample_info.drop("index", axis=1, inplace=True)
sample_info.head(len(sample_info))

In [ ]:
%%R -i sample_info -o Palette

# get pretty colours
Palette <- wes_palette(sample(names(wes_palettes), 1))

if (length(Palette) < length(unique(sample_info$category))){
    while (length(Palette) < length(unique(sample_info$category))){
        pal <- wes_palette(sample(names(wes_palettes), 1))
        Palette <- unique(c(Palette, pal))
    }
}

***
<br>


### Picard alignment summary stats

In [ ]:
# get gene_ids & gene_names for protein coding genes
gene_info = fetch_DataFrame('''select distinct b.contig, b.start, b.end, a.gene_id, 
            a.gene_name from gene_info a, geneset_all_gtf_genome_coordinates 
            b where a.gene_id = b.gene_id and transcript_biotype = "protein_coding"''', anndb)

In [ ]:
map_stats = fetch_DataFrame('''select PCT_ADAPTER, PCT_PF_READS_ALIGNED, READS_ALIGNED_IN_PAIRS, 
                                  TOTAL_READS, CATEGORY, sample_id from picardAlignmentSummary 
                                  where CATEGORY = "PAIR" ''', db)

map_stats["sample_id"] = map_stats["sample_id"].apply(lambda x: os.path.basename(x))
map_stats = pd.merge(map_stats, sample_info, how="inner", on="sample_id")

In [ ]:
%%R -i map_stats -w 1200 -h 700

get_legend <- function(a.gplot){ 
  tmp <- ggplot_gtable(ggplot_build(a.gplot)) 
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box") 
  legend <- tmp$grobs[[leg]] 
  return(legend)} 
                      
total_reads <- ggplot(map_stats, aes(y=TOTAL_READS/2, x=sample_id, fill=condition, shape=treatment)) + 
                geom_point(size=6) + 
                scale_y_continuous(limits=c(0, max(map_stats$TOTAL_READS)/2)) +
                geom_text_repel(aes(label=replicate), colour="black") +
                theme_notebook() +
                scale_shape_manual(values=c(21,22,23,24)) +
                scale_fill_manual(values=Palette) +
                scale_colour_manual(values=rev(c("red", "darkgray"))) +
                guides(fill=guide_legend(override.aes=list(shape=21))) +
                theme(axis.text.x=element_blank()) + labs(x="", y="Total Read Pairs")

mapped <- ggplot(map_stats, aes(y=READS_ALIGNED_IN_PAIRS/2, x=sample_id, fill=condition, shape=treatment)) + 
                geom_point(size=6) + 
                scale_y_continuous(limits=c(0, max(map_stats$READS_ALIGNED_IN_PAIRS)/2)) +
                geom_text_repel(aes(label=replicate), colour="black") +
                theme_notebook() +
                scale_shape_manual(values=c(21,22,23,24)) +
                scale_fill_manual(values=Palette) +
                scale_colour_manual(values=rev(c("red", "darkgray"))) +
                guides(fill=guide_legend(override.aes=list(shape=21))) +
                theme(axis.text.x=element_blank()) + labs(x="", y="Mapped Pairs")


pct_mapped <- ggplot(map_stats, aes(y=PCT_PF_READS_ALIGNED*100, x=sample_id, fill=condition, shape=treatment)) + 
                geom_point(size=6) + 
                scale_y_continuous(limits=c(0, 110)) +
                geom_text_repel(aes(label=replicate), colour="black") +
                theme_notebook() +
                scale_shape_manual(values=c(21,22,23,24)) +
                scale_fill_manual(values=Palette) +
                scale_colour_manual(values=rev(c("red", "darkgray"))) +
                guides(fill=guide_legend(override.aes=list(shape=21))) +
                theme(axis.text.x=element_blank()) + labs(x="", y="% Mapped in Pairs")


pct_adaptor <- ggplot(map_stats, aes(y=PCT_ADAPTER, x=sample_id,  fill=condition, shape=treatment)) + 
                geom_point(size=6) + 
                geom_text_repel(aes(label=replicate), colour="black") +
                theme_notebook() +
                scale_shape_manual(values=c(21,22,23,24)) +
                scale_fill_manual(values=Palette) +
                scale_colour_manual(values=rev(c("red", "darkgray"))) +
                guides(fill=guide_legend(override.aes=list(shape=21))) +
                theme(axis.text.x=element_blank()) + labs(x="", y="% adaptor")

                      
legend <- get_legend(total_reads + theme(legend.direction="horizontal"))

grid.arrange(top=textGrob("Mapping Stats", gp=gpar(fontfamily="Helvetica", fontface="bold", fontsize=23)), 
             total_reads + theme(legend.position="none"), mapped + theme(legend.position="none"), 
             pct_mapped + theme(legend.position="none"), pct_adaptor + theme(legend.position="none"), 
             ncol=2, nrow=2, bottom=legend)

***
<br>


### Picard RNA-seq metrics

#### Mapping annotations

In [ ]:
rna_map_stats_pct = fetch_DataFrame('''select PCT_USABLE_BASES, PCT_RIBOSOMAL_BASES, PCT_CODING_BASES, 
                                          PCT_UTR_BASES, PCT_INTRONIC_BASES, PCT_INTERGENIC_BASES, sample_id
                                          from picardRNAseqMetrics ''', db)

rna_map_stats_pct["PCT_RIBOSOMAL_BASES"] = rna_map_stats_pct["PCT_RIBOSOMAL_BASES"].apply(lambda x: float(0) if str(x) == "None" else x)
rna_map_stats_pct["sample_id"] = rna_map_stats_pct["sample_id"].apply(lambda x: os.path.basename(x))
rna_map_stats_pct = pd.merge(rna_map_stats_pct, sample_info, how="inner", on="sample_id")

In [ ]:
%%R -i rna_map_stats_pct -w 1200 

# melt <- melt(rna_map_stats, id_var=c("sample_id", "TF", "Day", "replicate", "LPS"))

# p1 <- ggplot(melt, aes(y=value, x=sample_id, fill=variable)) +
#            geom_bar(stat="identity",colour="black") +
#            scale_fill_manual(values=Palette) +
#            coord_flip() #+
# #            facet_wrap(~ facet)

melt2 <- melt(rna_map_stats_pct, id_var=c("sample_id", "condition", "treatment", "replicate"))

melt2$facet <- "feature / PF bases"
melt2$facet[melt2$variable=="PCT_USABLE_BASES"] <- "useable (mRNA / PF bases)"

p2 <- ggplot(melt2, aes(y=value, x=sample_id, fill=variable)) +
           geom_bar(stat="identity",colour="black") +
           theme_notebook() +
           scale_fill_manual(values=c(Palette, "darkseagreen3")) +
           coord_flip() +
           facet_wrap(~ facet) +
           labs(y="", x="") +
           scale_y_continuous(labels=percent)
        
p2

# Illumina sequencers perform an internal quality filtering procedure called chastity filter, 
# and reads that pass this filter are called PF for pass-filter. 
# According to Illumina, chastity is defined as the ratio of the brightest base intensity 
# divided by the sum of the brightest and second brightest base intensities. 
# Clusters of reads pass the filter if no more than 1 base call has a chastity value 
# below 0.6 in the first 25 cycles. This filtration process removes the least reliable 
# clusters from the image analysis results.

PF bases = pass-filter (Illumina internal quality filtering), only reads passing filter are counted

<br>

#### Strand orientation

In [ ]:
rna_map_strand = fetch_DataFrame('''select CORRECT_STRAND_READS, INCORRECT_STRAND_READS, 
                                  NUM_R1_TRANSCRIPT_STRAND_READS, NUM_R2_TRANSCRIPT_STRAND_READS, 
                                  sample_id from picardRNAseqMetrics ''', db)

rna_map_strand["sample_id"] = rna_map_strand["sample_id"].apply(lambda x: os.path.basename(x))
rna_map_strand = pd.merge(rna_map_strand, sample_info, how="inner", on="sample_id")

In [ ]:
%%R -i rna_map_strand -w 1200

melt <- melt(rna_map_strand, id_vars=c("sample_id", "condtion", "treatment", "replicate"))

p1 <- ggplot(melt, aes(y=value, x=sample_id, fill=variable)) + 
        geom_bar(stat="identity", colour="black") +
        scale_fill_manual(values=Palette) +
        theme_notebook() +
        coord_flip() +
        labs(y="", x="")
        
p1

- R1_TRANSCRIPT_STRAND_READS
    - The fraction of reads that support the model where R1 is on the strand of transcription and R2 is on the opposite strand. 
    - For unpaired reads, it is the fraction of reads that are on the transcription strand (out of all the reads).
- R2_TRANSCRIPT_STRAND_READS
    - The fraction of reads that support the model where R2 is on the strand of transcription and R1 is on the opposite strand. 
    - For unpaired reads, it is the fraction of reads that are on opposite strand than that of the the transcription strand (out of all the reads).

In [ ]:
rna_map_bias = fetch_DataFrame('''select MEDIAN_3PRIME_BIAS, MEDIAN_5PRIME_BIAS, 
                                  MEDIAN_5PRIME_TO_3PRIME_BIAS, MEDIAN_CV_COVERAGE,
                                  sample_id from picardRNAseqMetrics''', db)

rna_map_bias["sample_id"] = rna_map_bias["sample_id"].apply(lambda x: os.path.basename(x))
rna_map_bias = pd.merge(rna_map_bias, sample_info, how="inner", on="sample_id")
# rna_map_bias.head()

In [ ]:
%%R -i rna_map_bias -w 1200 -h 1800

melt <- melt(rna_map_bias[, c("MEDIAN_3PRIME_BIAS", "MEDIAN_5PRIME_BIAS", "sample_id", "condition", "treatment", "replicate")],
             id_vars=c("sample_id", "condition", "treatment", "replicate"))


p1 <- ggplot(rna_map_bias, aes(y=MEDIAN_CV_COVERAGE, x=sample_id, fill=condition)) +
        geom_bar(stat="identity", colour="black") +
        scale_fill_manual(values=Palette) +
        theme_notebook() +
        coord_flip() +
        labs(x="", caption="Ideal value = 0.\n",
            title="The median coefficient of variation (stdev/mean) for coverage values of the top 1,000 genes")
        
        
p2 <- ggplot(melt, aes(y=value, x=sample_id, fill=variable)) +
        geom_bar(stat="identity", colour="black") +
        scale_fill_manual(values=Palette) +
        theme_notebook() +
        coord_flip() +
        scale_y_continuous(labels=percent) +
        labs(y="", x="", title="The median 3 and 5 prime bias of the 1,000 most highly expressed transcripts",
            caption="\n")
        

p3 <- ggplot(rna_map_bias, aes(y=MEDIAN_5PRIME_TO_3PRIME_BIAS, x=sample_id, fill=condition)) +
        geom_bar(stat="identity", colour="black") +
        scale_fill_manual(values=Palette) +
        theme_notebook() +
        coord_flip() +
        labs(x="", caption="\n",
             title="The ratio of coverage at the 5 prime end to the 3 prime end based on the \n1,000 most highly expressed transcripts.")
        
        
grid.arrange(p1, p2, p3, ncol=1, nrow=3)

***
<br>

### Salmon TPMs

- Transcripts per million (TPM) were quantitated using Salmon and upper quantile normalised for between sample comparisons

In [ ]:
salmon_genes = fetch_DataFrame('''select * from salmon_genes''', db) # get data

# normalise to upper quantiles for between sample comparison
salmon_genes.index = salmon_genes["gene_id"]
salmon_genes.drop("gene_id", inplace=True, axis=1)
salmon_genes = salmon_genes.div(salmon_genes.quantile(q=0.75, axis=0), axis=1)
salmon_genes.reset_index(inplace=True)

# subset on protein coding genes
salmon_genes = pd.merge(salmon_genes, gene_info[["gene_id"]], how="inner", on="gene_id")

<br>

### Number of genes detected per sample 

In [ ]:
# Check how many genes are expressed across samples

def count_genes(df, sample_info, threshold):
    
    df = df.drop("gene_id", axis=1)
    
    expressed_genes = {}
    n=0
    for column in df:
        
        n=n+1
        if n == 1:
            tpm_count = pd.DataFrame(df.loc[df[column] > threshold][[column]].count(numeric_only=True))
        else:
            df2 = pd.DataFrame(df.loc[df[column] > threshold][[column]].count(numeric_only=True))
            tpm_count = tpm_count.append(df2)

    tpm_count["sample_id"] = tpm_count.index.values
    tpm_count.columns = ["no_genes", "sample_id"]
    tpm_count = pd.merge(tpm_count, sample_info, how="inner", on="sample_id")
    
    return tpm_count


tpm_count_0 = count_genes(salmon_genes, sample_info, 0)
tpm_count_1 = count_genes(salmon_genes, sample_info, 1)

In [ ]:
%%R -i tpm_count_0,tpm_count_1 -w 1200 -h 400

no_genes_0 <- ggplot(tpm_count_0, aes(y=no_genes, x=sample_id, fill=condition, shape=treatment)) + 
                geom_point(size=6) + scale_y_continuous(limits=c(0, 22000)) +
                geom_text(aes(label=replicate), colour="black", nudge_y=1800) +
                theme_notebook() +
                scale_shape_manual(values=c(21,22,23,24)) +
                scale_alpha_discrete(range=c(0.6,1)) +
                scale_fill_manual(values=Palette) +
                scale_colour_manual(values=c("black", "red3")) +
                theme(axis.text.x=element_blank()) + labs(x="", y="No. Genes (TPM >0)") +
                guides(fill=guide_legend(override.aes=list(shape=21))) +
                theme(legend.box="right")

no_genes_1 <- ggplot(tpm_count_1, aes(y=no_genes, x=sample_id, fill=condition, shape=treatment)) + 
                geom_point(size=6) + scale_y_continuous(limits=c(0, 22000)) +
                geom_text(aes(label=replicate), colour="black", nudge_y=1800) +
                theme_notebook() +
                scale_alpha_discrete(range=c(0.6,1)) +
                scale_shape_manual(values=c(21,22,23,24)) +
                scale_fill_manual(values=Palette) +
                scale_colour_manual(values=c("black", "red3")) +
                theme(axis.text.x=element_blank()) + labs(x="", y="No. Genes (TPM >1)") +
                guides(fill=guide_legend(override.aes=list(shape=21))) +
                theme(legend.box="right")

grid.arrange(no_genes_0, no_genes_1, ncol=2, nrow=1)

<br>

### Pearson correlation
- based on Manhattan distances between samples

#### For all detected genes:

In [ ]:
%%R -i salmon_genes -w 750 -h 650

# pearson correlation on all gene TPMs 

pal <- colorRampPalette(c("#3B9AB2", "#EBCC2A", "#F21A00"))(300)

plot_cor <- function(df) {
    
    df$gene_id <- NULL
    cm <- data.matrix(log2(df +1))
    m <- cor(cm, method="pearson", use="all")
    
    heatmap.2(m, 
      trace="none",
      key.xlab = "Pearson Correlation",
      key.ylab = "",
      col=pal,
      cexRow=1.5,
      cexCol=1.5,
      density.info=c("none"),
      mar=c(20,20),
      key.title=NA,
      lhei=c(0.3,2.2),
      lwid=c(0.6,2),
      key.par=list(cex.lab=2, cex.axis=1.5))}    

plot_cor(salmon_genes)

<br>

#### For top 5,000 genes:

In [ ]:
salmon_genes_top = salmon_genes.copy(deep=True)
salmon_genes_top["mean"] = salmon_genes_top.mean(axis=1)
salmon_genes_top = salmon_genes_top.sort_values("mean", ascending=False).head(5000)
salmon_genes_top.drop("mean", inplace=True, axis=1)

# salmon_genes_top.head()
# print(len(salmon_genes_top))

In [ ]:
%%R -i salmon_genes_top -w 750 -h 650

# pearson correlation on top 5000 gene TPMs  

plot_cor(salmon_genes_top)

***
<br>

### Replicate correlations

In [ ]:
# use sample information to get no. replicates & conditions
rep_pairs = sample_info.copy(deep=True)
rep_pairs = rep_pairs.pivot("category", "replicate", "sample_id").transpose()
rep_pairs.columns.name = None
rep_pairs.index.name = None

# report replicates to dict
reps = {}
for col in rep_pairs.columns:
    reps[col]=[rep_pairs[col].iloc[0], rep_pairs[col].iloc[1]]

# print(reps)

# log2 transfrom Salmon TPMs
salmon_genes.set_index("gene_id", inplace=True)
salmon_genes.fillna(0, inplace=True)
counts = np.log2(salmon_genes +1)

# get palette
pal = Palette
if len(rep_pairs.transpose()) > len(pal):
    import random
    
    extra_colours = sns.xkcd_rgb.keys() # all 954 colours
    pal2 = sns.xkcd_palette(extra_colours)
    pal = pal + random.sample(pal2, len(pal2))
    
sns.set(style="whitegrid", palette="muted")# set seaborn theme

# ignore seaborn deprication warning
import warnings
warnings.filterwarnings("ignore")

# use dict to subset df of normalised counts & plot rep correlations
n = 0
for key in reps:
    n = n + 1
    c = n - 1
    df = counts[reps[key]]
    df.columns = ["Rep1", "Rep2"]
    p = sns.jointplot(data=df, y="Rep1", x="Rep2", kind="reg", height=7, color=pal[c])
    plt.subplots_adjust(top=0.9)
    p.fig.suptitle(key) # add title
    p.annotate(stats.pearsonr)
    plt.show()
    plt.close()